<a href="https://colab.research.google.com/github/jihyoung-lee/stock_news_classifier/blob/main/news_title_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 requests pandas --quiet

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 검색할 키워드와 목표 수량 설정 + 라벨 정의
keywords = {
    "주가+상승": 1,
    "주가+급등": 1,
    "주가+하락": 0,
    "주가+폭락": 0
}
target_per_keyword = 25

# 결과 저장 리스트
news_titles = []
news_keywords = []
news_labels = []

# 키워드별 수집
for keyword, label in keywords.items():
    print(f"'{keyword}' 뉴스 검색 중...")

    page = 1
    collected = 0

    while collected < target_per_keyword:
        url = f"https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q={keyword}&p={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 뉴스 제목 추출
        titles = soup.select('div.c-item-content strong.tit-g.clamp-g a')

        if not titles:
            print(f"❗ '{keyword}' 관련 뉴스가 더 이상 없습니다.")
            break

        for tag in titles:
            title = tag.get_text(strip=True)
            if title not in news_titles:  # 중복 제거
                news_titles.append(title)
                news_keywords.append(keyword)
                news_labels.append(label)
                collected += 1
            if collected >= target_per_keyword:
                break

        page += 1
        time.sleep(0.5)

# 데이터프레임 생성
df = pd.DataFrame({
    "keyword": news_keywords,
    "title": news_titles,
    "label": news_labels
})

# CSV 저장
csv_filename = "stock_news_labeled_auto.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\n✅ 총 {len(news_titles)}개 뉴스 수집 및 라벨링 완료!")
print(f"✅ '{csv_filename}' 파일로 저장 완료!")

🔍 '주가+상승' 뉴스 검색 중...
🔍 '주가+급등' 뉴스 검색 중...
🔍 '주가+하락' 뉴스 검색 중...
🔍 '주가+폭락' 뉴스 검색 중...

✅ 총 100개 뉴스 수집 및 라벨링 완료!
✅ 'stock_news_labeled_auto.csv' 파일로 저장 완료!
🔍 '주가+상승' 뉴스 검색 중...
🔍 '주가+급등' 뉴스 검색 중...
🔍 '주가+하락' 뉴스 검색 중...
🔍 '주가+폭락' 뉴스 검색 중...

✅ 총 100개 뉴스 수집 및 라벨링 완료!
✅ 'stock_news_labeled_auto.csv' 파일로 저장 완료!


In [ ]:
# 기존 수집된 뉴스 수
existing_count = len(news_titles)

# 목표 수량 설정
total_target = 300

# 부족한 만큼 추가로 크롤링
if existing_count < total_target:
    additional_needed = total_target - existing_count
    print(f"뉴스 {additional_needed}개 더 수집해야 합니다.")

    page = page  # 기존 마지막 페이지부터 이어서
    while len(news_titles) < total_target:
        url = f"https://search.daum.net/search?w=news&q=주가+상승&p={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        titles = soup.select('div.c-item-content strong.tit-g.clamp-g a')

        if not titles:
            print("❗ 추가 뉴스가 없습니다.")
            break

        for tag in titles:
            title = tag.get_text(strip=True)
            if title not in news_titles:  # 중복 제거
                news_titles.append(title)
                news_keywords.append('주가 상승')
                news_labels.append(1)  # 라벨도 추가
            if len(news_titles) >= total_target:
                break

        page += 1
        time.sleep(0.5)